In [ ]:
import os
import random
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set the path to your image dataset
data_dir = '/content/drive/MyDrive/fulhaus/Dataset_refined'

# Define the number of classes in your custom dataset
num_classes = 3

# Define the input size of the images for the model
input_size = (224, 224)

# Define the batch size and number of epochs for training
batch_size = 32
epochs = 10

# Define the ratio of images to use for validation
validation_split = 0.2

# Create a list of all the image filenames in the dataset directory
filenames = os.listdir(data_dir)

# Shuffle the list of filenames
random.shuffle(filenames)

# Calculate the number of images to use for validation
num_validation = int(len(filenames) * validation_split)

# Split the filenames into training and validation sets
train_filenames = filenames[num_validation:]
valid_filenames = filenames[:num_validation]

# Define the data augmentation configuration for training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Define the data augmentation configuration for validation data
valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Generate training data batches
train_data = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Generate validation data batches
valid_data = valid_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Load the pre-trained ResNet50 model with ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_size+(3,))

# Add custom classification layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define the final classification model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the pre-trained layers of the model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data
model.fit(train_data, epochs=epochs, validation_data=valid_data)

# Save the trained model to a file
model.save('/content/drive/MyDrive/fulhaus/resnet.h5')

# Evaluate the model on the validation data
score = model.evaluate(valid_data)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])


Found 300 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


Epoch 1/10
10/10 [==============================] - 176s 17s/step - loss: 1.5969 - accuracy: 0.5933 - val_loss: 0.2915 - val_accuracy: 0.9167
Epoch 2/10
10/10 [==============================] - 22s 2s/step - loss: 0.4841 - accuracy: 0.8567 - val_loss: 0.2070 - val_accuracy: 0.9333
Epoch 3/10
10/10 [==============================] - 22s 2s/step - loss: 0.2039 - accuracy: 0.9367 - val_loss: 0.0307 - val_accuracy: 0.9833
Epoch 4/10
10/10 [==============================] - 22s 2s/step - loss: 0.0927 - accuracy: 0.9700 - val_loss: 0.0088 - val_accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 23s 2s/step - loss: 0.0343 - accuracy: 0.9867 - val_loss: 0.0040 - val_accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 21s 2s/step - loss: 0.0499 - accuracy: 0.9833 - val_loss: 0.0054 - val_accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 22s 2s/step - loss: 0.0633 - accuracy: 0.9800 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 8/10
10/10 [